In [11]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks
import hashlib
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ImageForgeryDetection:
    def __init__(self, dataset_dir, img_size=(128, 128)):
        """
        Initialize the ImageForgeryDetection class.
        
        :param dataset_dir: Path to the dataset directory.
        :param img_size: Tuple specifying the size to which images will be resized.
        """
        self.dataset_dir = dataset_dir
        self.img_size = img_size
        self.model = None

    def load_and_preprocess_data(self):
        """
        Load and preprocess the dataset.
        
        :return: Tuple of (train_images, test_images, train_labels, test_labels)
        """
        logging.info("Loading and preprocessing dataset...")

        images = []
        labels = []

        # Check if dataset directory exists
        if not os.path.exists(self.dataset_dir):
            logging.error(f"Dataset directory {self.dataset_dir} does not exist.")
            return None, None, None, None

        # Assuming dataset has two folders: 'original' and 'tampered'
        for label in ['original', 'tampered']:
            folder_path = os.path.join(self.dataset_dir, label)
            if not os.path.exists(folder_path):
                logging.error(f"Folder {folder_path} does not exist.")
                continue

            for filename in os.listdir(folder_path):
                img_path = os.path.join(folder_path, filename)
                img = cv2.imread(img_path)
                if img is None:
                    logging.warning(f"Failed to load image: {img_path}")
                    continue

                img = cv2.resize(img, self.img_size)  # Resize image
                img = img / 255.0  # Normalize image
                images.append(img)
                labels.append(1 if label == 'tampered' else 0)  # 0 for original, 1 for tampered

        # Convert to numpy arrays
        images = np.array(images)
        labels = np.array(labels)

        # Ensure there are enough samples for splitting
        if len(images) == 0:
            logging.error("No valid images found in the dataset.")
            return None, None, None, None

        if len(np.unique(labels)) < 2:
            logging.error("Dataset does not contain enough classes for classification.")
            return None, None, None, None

        # Split data into training and testing sets
        train_images, test_images, train_labels, test_labels = train_test_split(
            images, labels, test_size=0.2, random_state=42, stratify=labels
        )

        logging.info(f"Dataset loaded: {len(train_images)} train samples, {len(test_images)} test samples.")
        return train_images, test_images, train_labels, test_labels

    def build_model(self):
        """
        Build a Convolutional Neural Network (CNN) model.
        """
        logging.info("Building CNN model...")
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*self.img_size, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),  # Add dropout for regularization
            layers.Dense(1, activation='sigmoid')  # Binary classification (original or tampered)
        ])

        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        self.model.summary()
        logging.info("CNN model built successfully.")

    def train_model(self, train_images, train_labels, test_images, test_labels, epochs=10, batch_size=32):
        """
        Train the CNN model.
        
        :param train_images: Training images.
        :param train_labels: Training labels.
        :param test_images: Testing images.
        :param test_labels: Testing labels.
        :param epochs: Number of training epochs.
        :param batch_size: Batch size for training.
        """
        logging.info("Training the model...")

        # Define callbacks for early stopping and model checkpoint
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

        history = self.model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size,
                                 validation_data=(test_images, test_labels),
                                 callbacks=[early_stopping, checkpoint])

        logging.info("Model training completed.")

    def evaluate_model(self, test_images, test_labels):
        """
        Evaluate the trained model on the test dataset.
        
        :param test_images: Testing images.
        :param test_labels: Testing labels.
        :return: Tuple of (test_loss, test_accuracy)
        """
        logging.info("Evaluating the model...")
        test_loss, test_accuracy = self.model.evaluate(test_images, test_labels)
        logging.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")
        return test_loss, test_accuracy

    def generate_image_hash(self, image_path):
        """
        Generate SHA-256 hash of an image file.
        
        :param image_path: Path to the image file.
        :return: SHA-256 hash of the image.
        """
        logging.info(f"Generating hash for image: {image_path}")
        with open(image_path, "rb") as f:
            img_bytes = f.read()
        return hashlib.sha256(img_bytes).hexdigest()

    def verify_image_integrity(self, original_image_path, uploaded_image_path):
        """
        Verify the integrity of an uploaded image using its hash.
        
        :param original_image_path: Path to the original image.
        :param uploaded_image_path: Path to the uploaded image.
        :return: Boolean indicating whether the uploaded image matches the original.
        """
        logging.info("Verifying image integrity...")
        original_hash = self.generate_image_hash(r"C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg")
        uploaded_hash = self.generate_image_hash(r"C:\Users\saniu\Machine Learning 2025\Block Chain\saz - Copy.jpg")
        return original_hash == uploaded_hash


if __name__ == "__main__":
    # Replace with your dataset directory path
    dataset_directory = r"C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg"  

    detector = ImageForgeryDetection(dataset_dir=dataset_directory)

    # Load and preprocess the dataset
    train_images, test_images, train_labels, test_labels = detector.load_and_preprocess_data()

    if train_images is not None and test_images is not None:
        # Build the CNN model
        detector.build_model()

        # Train the model
        detector.train_model(train_images, train_labels, test_images, test_labels, epochs=10, batch_size=32)

        # Evaluate the model
        detector.evaluate_model(test_images, test_labels)
    else:
        logging.error("Dataset loading failed. Please check your dataset.")


2025-02-08 11:51:33,210 - INFO - Loading and preprocessing dataset...
2025-02-08 11:51:33,211 - ERROR - Folder C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg\original does not exist.
2025-02-08 11:51:33,212 - ERROR - Folder C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg\tampered does not exist.
2025-02-08 11:51:33,213 - ERROR - No valid images found in the dataset.
2025-02-08 11:51:33,213 - ERROR - Dataset loading failed. Please check your dataset.


In [15]:
import os
import hashlib
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ImageIntegrityChecker:
    def __init__(self):
        """
        Initialize the ImageIntegrityChecker class.
        """
        pass

    def generate_image_hash(self, image_path):
        """
        Generate SHA-256 hash of an image file.

        :param image_path: Path to the image file.
        :return: SHA-256 hash of the image.
        """
        if not os.path.exists(image_path):
            logging.error(f"Image file not found: {image_path}")
            return None

        logging.info(f"Generating hash for image: {image_path}")
        with open(image_path, "rb") as f:
            img_bytes = f.read()
        return hashlib.sha256(img_bytes).hexdigest()

    def verify_image_integrity(self, original_image_path, uploaded_image_path):
        """
        Verify the integrity of an uploaded image using its hash.

        :param original_image_path: Path to the original image.
        :param uploaded_image_path: Path to the uploaded image.
        :return: Boolean indicating whether the uploaded image matches the original.
        """
        original_hash = self.generate_image_hash(original_image_path)
        uploaded_hash = self.generate_image_hash(uploaded_image_path)

        if original_hash is None or uploaded_hash is None:
            logging.error("One or both images could not be hashed.")
            return False

        logging.info(f"Original Image Hash: {original_hash}")
        logging.info(f"Uploaded Image Hash: {uploaded_hash}")

        if original_hash == uploaded_hash:
            logging.info("The uploaded image matches the original image.")
            return True
        else:
            logging.warning("The uploaded image does NOT match the original image.")
            return False


if __name__ == "__main__":
    # Initialize the ImageIntegrityChecker class
    checker = ImageIntegrityChecker()

    # Paths to the original and uploaded images
    original_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg"  # Replace with your original image path
    uploaded_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\saz1.jpeg"  # Replace with your uploaded image path

    # Verify image integrity
    is_valid = checker.verify_image_integrity(original_image_path, uploaded_image_path)

    # Print result
    print(f"Is the uploaded image valid? {'Yes' if is_valid else 'No'}")

2025-02-08 11:57:18,336 - INFO - Generating hash for image: C:\Users\saniu\Machine Learning 2025\Block Chain\saz.jpg
2025-02-08 11:57:18,337 - ERROR - Image file not found: C:\Users\saniu\Machine Learning 2025\Block Chain\saz1.jpeg
2025-02-08 11:57:18,337 - ERROR - One or both images could not be hashed.


Is the uploaded image valid? No


In [3]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ImageForgeryDetection:
    def __init__(self, dataset_dir, img_size=(128, 128), batch_size=32):
        """
        Initialize the ImageForgeryDetection class.

        :param dataset_dir: Path to the dataset directory.
        :param img_size: Tuple specifying the size to which images will be resized.
        :param batch_size: Batch size for data processing.
        """
        self.dataset_dir = dataset_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.model = None

    def load_dataset(self):
        """
        Load and preprocess the dataset using tf.data.Dataset API.

        :return: Tuple of (train_dataset, test_dataset)
        """
        logging.info("Loading and preprocessing dataset...")

        # Define image paths and labels
        original_folder = os.path.join(self.dataset_dir, 'original')
        tampered_folder = os.path.join(self.dataset_dir, 'tampered')

        if not os.path.exists(original_folder) or not os.path.exists(tampered_folder):
            logging.error(f"Original or Tampered folder not found in {self.dataset_dir}.")
            return None, None

        original_images = [os.path.join(original_folder, f) for f in os.listdir(original_folder)]
        tampered_images = [os.path.join(tampered_folder, f) for f in os.listdir(tampered_folder)]

        all_image_paths = original_images + tampered_images
        all_labels = [0] * len(original_images) + [1] * len(tampered_images)

        # Shuffle the data
        combined = list(zip(all_image_paths, all_labels))
        np.random.shuffle(combined)
        all_image_paths, all_labels = zip(*combined)

        # Convert to numpy arrays
        all_image_paths = np.array(all_image_paths)
        all_labels = np.array(all_labels)

        # Split into training and testing sets
        train_paths, test_paths, train_labels, test_labels = train_test_split(
            all_image_paths, all_labels, test_size=0.2, random_state=42, stratify=all_labels
        )

        # Define a function to load and preprocess images
        def load_and_preprocess_image(image_path, label):
            img = tf.io.read_file(image_path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, self.img_size)
            img = tf.cast(img, tf.float32) / 255.0  # Normalize image
            return img, label

        # Create tf.data.Dataset pipelines
        train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
        train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        train_dataset = train_dataset.shuffle(buffer_size=1000).batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

        test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
        test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        test_dataset = test_dataset.batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

        logging.info(f"Dataset loaded: {len(train_paths)} train samples, {len(test_paths)} test samples.")
        return train_dataset, test_dataset

    def build_model(self):
        """
        Build a Convolutional Neural Network (CNN) model.
        """
        logging.info("Building CNN model...")
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*self.img_size, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),  # Add dropout for regularization
            layers.Dense(1, activation='sigmoid')  # Binary classification (original or tampered)
        ])

        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        self.model.summary()
        logging.info("CNN model built successfully.")

    def train_model(self, train_dataset, test_dataset, epochs=10):
        """
        Train the CNN model.

        :param train_dataset: Training dataset.
        :param test_dataset: Testing dataset.
        :param epochs: Number of training epochs.
        """
        logging.info("Training the model...")

        # Define callbacks for early stopping and model checkpoint
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

        history = self.model.fit(train_dataset, epochs=epochs,
                                 validation_data=test_dataset,
                                 callbacks=[early_stopping, checkpoint])

        logging.info("Model training completed.")

    def evaluate_model(self, test_dataset):
        """
        Evaluate the trained model on the test dataset.

        :param test_dataset: Testing dataset.
        :return: Tuple of (test_loss, test_accuracy)
        """
        logging.info("Evaluating the model...")
        test_loss, test_accuracy = self.model.evaluate(test_dataset)
        logging.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")
        return test_loss, test_accuracy


if __name__ == "__main__":
    # Replace with your dataset directory path
    dataset_directory = r"C:\Users\saniu\Machine Learning 2025\Block Chain"  # Update this path
    detector = ImageForgeryDetection(dataset_dir=dataset_directory, img_size=(128, 128), batch_size=32)

    # Load the dataset
    train_dataset, test_dataset = detector.load_dataset()

    if train_dataset is not None and test_dataset is not None:
        # Build the CNN model
        detector.build_model()

        # Train the model
        detector.train_model(train_dataset, test_dataset, epochs=10)

        # Evaluate the model
        detector.evaluate_model(test_dataset)
    else:
        logging.error("Dataset loading failed. Please check your dataset.")

2025-02-08 12:21:01,314 - INFO - Loading and preprocessing dataset...
2025-02-08 12:21:01,716 - INFO - Dataset loaded: 5776 train samples, 1445 test samples.
2025-02-08 12:21:01,717 - INFO - Building CNN model...
C:\Users\saniu\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

...

2025-02-08 12:22:14,825 - WARNING - You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. 


181/181 ━━━━━━━━━━━━━━━━━━━━ 73s 370ms/step - accuracy: 0.9819 - loss: 0.0575 - val_accuracy: 1.0000 - val_loss: 4.0959e-10
Epoch 2/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 63s 324ms/step - accuracy: 0.9999 - loss: 0.0050 - val_accuracy: 1.0000 - val_loss: 5.8792e-05
Epoch 3/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 62s 319ms/step - accuracy: 0.9998 - loss: 0.0074 - val_accuracy: 1.0000 - val_loss: 1.8772e-07
Epoch 4/10
181/181 ━━━━━━━━━━━━━━━━━━━━ 62s 320ms/step - accuracy: 0.9997 - loss: 0.0054 - val_accuracy: 1.0000 - val_loss: 1.2520e-06


2025-02-08 12:25:21,435 - INFO - Model training completed.
2025-02-08 12:25:21,435 - INFO - Evaluating the model...


46/46 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 1.0000 - loss: 2.3219e-10


2025-02-08 12:25:26,960 - INFO - Test Loss: 0.0000, Test Accuracy: 100.00%


In [7]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, callbacks
import hashlib
import logging
import json
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class ImageForgeryDetection:
    def __init__(self, dataset_dir, img_size=(128, 128), batch_size=32):
        """
        Initialize the ImageForgeryDetection class.

        :param dataset_dir: Path to the dataset directory.
        :param img_size: Tuple specifying the size to which images will be resized.
        :param batch_size: Batch size for data processing.
        """
        self.dataset_dir = dataset_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.model = None

    def load_dataset(self):
        """
        Load and preprocess the dataset using tf.data.Dataset API.

        :return: Tuple of (train_dataset, test_dataset)
        """
        logging.info("Loading and preprocessing dataset...")

        # Define image paths and labels
        original_folder = os.path.join(self.dataset_dir, 'original')
        tampered_folder = os.path.join(self.dataset_dir, 'tampered')

        if not os.path.exists(original_folder) or not os.path.exists(tampered_folder):
            logging.error(f"Original or Tampered folder not found in {self.dataset_dir}.")
            return None, None

        original_images = [os.path.join(original_folder, f) for f in os.listdir(original_folder)]
        tampered_images = [os.path.join(tampered_folder, f) for f in os.listdir(tampered_folder)]

        all_image_paths = original_images + tampered_images
        all_labels = [0] * len(original_images) + [1] * len(tampered_images)

        # Shuffle the data
        combined = list(zip(all_image_paths, all_labels))
        np.random.shuffle(combined)
        all_image_paths, all_labels = zip(*combined)

        # Convert to numpy arrays
        all_image_paths = np.array(all_image_paths)
        all_labels = np.array(all_labels)

        # Split into training and testing sets
        train_paths, test_paths, train_labels, test_labels = train_test_split(
            all_image_paths, all_labels, test_size=0.2, random_state=42, stratify=all_labels
        )

        # Define a function to load and preprocess images
        def load_and_preprocess_image(image_path, label):
            img = tf.io.read_file(image_path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, self.img_size)
            img = tf.cast(img, tf.float32) / 255.0  # Normalize image
            return img, label

        # Create tf.data.Dataset pipelines
        train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
        train_dataset = train_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        train_dataset = train_dataset.shuffle(buffer_size=1000).batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

        test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
        test_dataset = test_dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        test_dataset = test_dataset.batch(self.batch_size).prefetch(tf.data.AUTOTUNE)

        logging.info(f"Dataset loaded: {len(train_paths)} train samples, {len(test_paths)} test samples.")
        return train_dataset, test_dataset

    def build_model(self):
        """
        Build a Convolutional Neural Network (CNN) model.
        """
        logging.info("Building CNN model...")
        self.model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*self.img_size, 3)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),  # Add dropout for regularization
            layers.Dense(1, activation='sigmoid')  # Binary classification (original or tampered)
        ])

        self.model.compile(optimizer='adam',
                           loss='binary_crossentropy',
                           metrics=['accuracy'])

        self.model.summary()
        logging.info("CNN model built successfully.")

    def train_model(self, train_dataset, test_dataset, epochs=10):
        """
        Train the CNN model.

        :param train_dataset: Training dataset.
        :param test_dataset: Testing dataset.
        :param epochs: Number of training epochs.
        """
        logging.info("Training the model...")

        # Define callbacks for early stopping and model checkpoint
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        checkpoint = callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

        history = self.model.fit(train_dataset, epochs=epochs,
                                 validation_data=test_dataset,
                                 callbacks=[early_stopping, checkpoint])

        logging.info("Model training completed.")

    def evaluate_model(self, test_dataset):
        """
        Evaluate the trained model on the test dataset.

        :param test_dataset: Testing dataset.
        :return: Tuple of (test_loss, test_accuracy)
        """
        logging.info("Evaluating the model...")
        test_loss, test_accuracy = self.model.evaluate(test_dataset)
        logging.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")
        return test_loss, test_accuracy

    def predict_image(self, image_path):
        """
        Predict whether an image is original or tampered.

        :param image_path: Path to the image file.
        :return: Prediction result (0 for original, 1 for tampered).
        """
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, self.img_size)
        img = tf.cast(img, tf.float32) / 255.0  # Normalize image
        img = tf.expand_dims(img, axis=0)  # Add batch dimension

        prediction = self.model.predict(img)
        return prediction[0][0]
    @staticmethod
    def generate_image_hash(image_path):
        """
        Generate SHA-256 hash of an image file.

        :param image_path: Path to the image file.
        :return: SHA-256 hash of the image.
        """
        with open(image_path, "rb") as f:
            img_bytes = f.read()
        image_hash = hashlib.sha256(img_bytes).hexdigest()
        logging.info(f"Image Hash: {image_hash}")
        return image_hash  

class Blockchain:
    def __init__(self):
        """
        Initialize the Blockchain class.
        """
        self.chain = []
        self.create_block(proof=1, previous_hash='0')

    def create_block(self, proof, previous_hash):
        """
        Create a new block in the blockchain.

        :param proof: Proof of work.
        :param previous_hash: Hash of the previous block.
        :return: New block.
        """
        block = {
            'index': len(self.chain) + 1,
            'timestamp': str(time.time()),
            'proof': proof,
            'previous_hash': previous_hash,
            'data': []  # Data field to store image hashes
        }
        self.chain.append(block)
        return block

    def get_previous_block(self):
        """
        Get the previous block in the blockchain.

        :return: Previous block.
        """
        return self.chain[-1]

    def proof_of_work(self, previous_proof):
        """
        Simple proof of work algorithm.

        :param previous_proof: Proof of the previous block.
        :return: New proof.
        """
        new_proof = 1
        check_proof = False
        while check_proof is False:
            hash_operation = hashlib.sha256(str(new_proof ** 2 - previous_proof ** 2).encode()).hexdigest()
            if hash_operation[:4] == '0000':
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def hash(self, block):
        """
        Hash a block.

        :param block: Block to be hashed.
        :return: SHA-256 hash of the block.
        """
        encoded_block = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def add_image_to_blockchain(self, image_path):
        """
        Add an image to the blockchain by storing its hash.

        :param image_path: Path to the image file.
        """
        previous_block = self.get_previous_block()
        previous_proof = previous_block['proof']
        proof = self.proof_of_work(previous_proof)
        previous_hash = self.hash(previous_block)

        # Generate image hash
        with open(image_path, "rb") as f:
            img_bytes = f.read()
        image_hash = hashlib.sha256(img_bytes).hexdigest()

        # Add image hash to the block
        new_block = self.create_block(proof, previous_hash)
        new_block['data'].append({'image_hash': image_hash, 'path': image_path})
        logging.info(f"Image added to blockchain: {image_path}")


if __name__ == "__main__":
    # Replace with your dataset directory path
    dataset_directory = r"C:\Users\saniu\Machine Learning 2025\Block Chain"  # Update this path
    detector = ImageForgeryDetection(dataset_dir=dataset_directory, img_size=(128, 128), batch_size=32)

    # Load the dataset
    train_dataset, test_dataset = detector.load_dataset()

    if train_dataset is not None and test_dataset is not None:
        # Build the CNN model
        detector.build_model()

        # Train the model
        detector.train_model(train_dataset, test_dataset, epochs=10)

        # Evaluate the model
        detector.evaluate_model(test_dataset)

        # Initialize blockchain
        blockchain = Blockchain()

        # Example: Add an image to the blockchain
        example_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\original\1 (3).jpg"
        blockchain.add_image_to_blockchain(example_image_path)

        # Example: Predict if an image is tampered
        test_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\tampered\saz1 copy.jpg"
        image_hash = ImageForgeryDetection.generate_image_hash(test_image_path)
        logging.info(f"Hash of '{test_image_path}': {image_hash}")
        prediction = detector.predict_image(test_image_path)
        if prediction < 0.5:
            logging.info(f"Image '{test_image_path}' is Original.")
        else:
            logging.info(f"Image '{test_image_path}' is Tampered.")
    else:
        logging.error("Dataset loading failed. Please check your dataset.")

2025-02-08 13:06:40,101 - INFO - Loading and preprocessing dataset...
2025-02-08 13:06:40,156 - INFO - Dataset loaded: 5778 train samples, 1445 test samples.
2025-02-08 13:06:40,158 - INFO - Building CNN model...
C:\Users\saniu\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

...

2025-02-08 13:13:55,793 - INFO - Model training completed.
2025-02-08 13:13:55,794 - INFO - Evaluating the model...


46/46 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - accuracy: 0.9997 - loss: 0.0029


2025-02-08 13:14:01,227 - INFO - Test Loss: 0.0061, Test Accuracy: 99.93%
2025-02-08 13:14:01,230 - INFO - Image added to blockchain: C:\Users\saniu\Machine Learning 2025\Block Chain\original\1 (3).jpg
2025-02-08 13:14:01,232 - INFO - Image Hash: f5764ee04c65f80bef06633e53e663d5fbb7fdaa1f4b9ee34212b5f1988b1109
2025-02-08 13:14:01,232 - INFO - Hash of 'C:\Users\saniu\Machine Learning 2025\Block Chain\tampered\saz1 copy.jpg': f5764ee04c65f80bef06633e53e663d5fbb7fdaa1f4b9ee34212b5f1988b1109


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


2025-02-08 13:14:01,380 - INFO - Image 'C:\Users\saniu\Machine Learning 2025\Block Chain\tampered\saz1 copy.jpg' is Original.


In [9]:
import hashlib

def generate_image_hash(image_path):
    with open(image_path, "rb") as f:
        img_bytes = f.read()
    return hashlib.sha256(img_bytes).hexdigest()

original_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\original\1 (3).jpg"
tampered_image_path = r"C:\Users\saniu\Machine Learning 2025\Block Chain\tampered\saz1 copy.jpg"

original_hash = generate_image_hash(original_image_path)
tampered_hash = generate_image_hash(tampered_image_path)

print(f"Original Image Hash: {original_hash}")
print(f"Tampered Image Hash: {tampered_hash}")
print(f"Do the hashes match? {'Yes' if original_hash == tampered_hash else 'No'}")

Original Image Hash: 8588b218577b0022b1a1fabe4cd9dd04c7951b2de668337355ba6a61538cd9cc
Tampered Image Hash: f5764ee04c65f80bef06633e53e663d5fbb7fdaa1f4b9ee34212b5f1988b1109
Do the hashes match? No
